In [ ]:
import pandas as pd
import os
import json

In [ ]:
df = pd.read_excel("results/Perguntas.xlsx", sheet_name="finais")

In [ ]:
df = df.dropna(subset=["Resposta"])

In [ ]:
df.head()

In [ ]:
subs = ["Pergunta","Intenção"]
df2 = df[subs]
df2 = df2.rename(columns={"Pergunta": "examples", "Intenção": "intent"})

In [ ]:
skill_path = "results/skill-Amazônia-Azul.json"
with open(skill_path, "r", encoding="utf-8") as f:
    skill = json.load(f)
skill

In [ ]:
intents = skill["intents"]
intents

In [ ]:
new_intents = df2.to_dict(orient="records")
new_intents = [{"intent": i["intent"], "examples": [{"text": i["examples"]}], "description": ""} for i in new_intents]
new_intents

In [ ]:
skill["intents"] = new_intents
skill

In [ ]:
skill_root, skill_ext = os.path.splitext(skill_path)
new_skill_path = f"{skill_root}2{skill_ext}"
with open(new_skill_path, "w", encoding="utf-8") as f:
    json.dump(skill, f, ensure_ascii=False)